In [18]:
import pandas as pd
import numpy as np

# Corregir la ruta del archivo
df = pd.read_parquet("/home/donsson/proyectos/API/historico_ventas_1_año.parquet")
df_p = pd.read_parquet("/home/donsson/proyectos/API/costo_productos.parquet")


In [19]:
import re
import unicodedata

# Diccionario de códigos a sucursales
mapa_codigos = {
    "FCAL": "CALI",
    "FMDE": "MEDELLIN",
    "FBOG": "BOGOTA",
    "FCTG": "CARTAGENA",
    "FBAQ": "BARRANQUILLA",
    "FVAL":"VALLADOLID"
}


# Equivalencias para normalizar nombres truncados o mal escritos
mapa_equivalencias = {
    "MEDELLIN": "MEDELLIN",
    "MEDELLI": "MEDELLIN",
    "MEDELL": "MEDELLIN",
    "MEDELI": "MEDELLIN",
    "CALI": "CALI",
    "BARRANQUILLA": "BARRANQUILLA",
    "BOGOTA": "BOGOTA",
    "CARTAGENA": "CARTAGENA",
    "VALLADOLID": "VALLADOLID"
}

def normalizar(texto):
    """Quita tildes y pasa a mayúsculas"""
    texto = unicodedata.normalize("NFKD", texto)
    texto = "".join([c for c in texto if not unicodedata.combining(c)])
    return texto.upper()

def extraer_sucursal(nombre):
    if not isinstance(nombre, str):
        return "VENDEDOR EXTERNO"
    
    sucursal = None
    
    # 1) Buscar "Mostrador ..."
    match = re.search(r"Mostrador\s+([A-Za-z0-9\s]+)", nombre, re.IGNORECASE)
    if match:
        sucursal = match.group(1).strip()
    else:
        # 2) Buscar "Calle" o "Cota"
        match2 = re.search(r"(Calle\s+\d+|Cota)", nombre, re.IGNORECASE)
        if match2:
            sucursal = match2.group(1).strip()
        else:
            # 3) Buscar prefijo de código
            for prefijo, ciudad in mapa_codigos.items():
                if nombre.upper().startswith(prefijo):
                    return ciudad
            return "VENDEDOR EXTERNO"
    
    # Normalizar texto
    sucursal = normalizar(sucursal)
    
    # Limpiar T1, T2, T3 al final
    sucursal = re.sub(r"\s*T\d+$", "", sucursal).strip()
    
    # Aplicar equivalencias
    sucursal = mapa_equivalencias.get(sucursal, sucursal)
    
    return sucursal

# Aplicar al dataframe
df["Sucursal"] = df["invoice_name"].apply(extraer_sucursal)

In [20]:
df = pd.merge(df,df_p,on='product_name',how="left")
# --- Dejar solo facturas únicas ---
df = df.drop_duplicates(subset=["invoice_id_num"]).copy() 


df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 136608 entries, 0 to 4590671
Data columns (total 10 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   price_subtotal           136608 non-null  float64       
 1   id                       136608 non-null  int64         
 2   quantity                 136608 non-null  float64       
 3   product_id_num           136608 non-null  int64         
 4   product_name             136608 non-null  object        
 5   invoice_id_num           136608 non-null  int64         
 6   invoice_name             136608 non-null  object        
 7   date_invoice             136608 non-null  datetime64[ns]
 8   Sucursal                 136608 non-null  object        
 9   producto_costo_unitario  136399 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(3), object(3)
memory usage: 11.5+ MB


In [26]:
df = df[df["Sucursal"]!= "VENDEDOR EXTERNO"]

df = df[["quantity","product_name","date_invoice","Sucursal","producto_costo_unitario"]]

# Aseguramos que la columna sea datetime (por si acaso)
df["date_invoice"] = pd.to_datetime(df["date_invoice"])

# Tomamos la fecha máxima del DF
fecha_max = df["date_invoice"].max()

# Calculamos el límite de un año atrás
fecha_min = fecha_max - pd.Timedelta(weeks=156)

# Filtramos
df_1_año= df[df["date_invoice"] >= fecha_min]

print(df_1_año.shape)
print(df_1_año["date_invoice"].min(), df_1_año["date_invoice"].max())


df_1_año.head(5)


(108783, 5)
2022-09-13 20:25:02 2025-09-09 20:20:48


,quantity,product_name,date_invoice,Sucursal,producto_costo_unitario
0,1.0,[BHS00073125] GS073 FILTRO HIDRAULICO VARIOS (...,2025-09-09 20:20:48,BARRANQUILLA,42227.53
9,1.0,[BWS00078125] GS078A4 FILTRO REFRIGERACION CUM...,2025-09-09 20:18:54,MEDELLIN,27840.29
45,3.0,[BLS00320125] GS320 FILTRO ACEITE INGER S. RAN...,2025-09-09 20:17:48,CALI,58205.33
126,1.0,[DAB02902025] DA2902 FILTRO AIRE LISTER PETTER...,2025-09-09 20:11:38,MEDELLIN,19218.23
144,2.0,[DAB14570025] DA4570A FILTRO AIRE 2_ DONSSON -...,2025-09-09 20:10:02,CALLE 6,9774.98


In [27]:
prueba = df_1_año[(df["Sucursal"]=="BARRANQUILLA")&( df_1_año["product_name"].str.contains("DAB02570025"))]
#prueba.to_excel("mirar.xlsx")
ventas_semana = (
    prueba
    .groupby(pd.Grouper(key="date_invoice", freq="W"))["quantity"]
    .sum()
    .reset_index()
)

ventas_mes = (
    prueba
    .groupby(pd.Grouper(key="date_invoice", freq="M"))["quantity"]
    .sum()
    .reset_index()
)


/tmp/ipykernel_61671/3414202852.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  prueba = df_1_año[(df["Sucursal"]=="BARRANQUILLA")&( df_1_año["product_name"].str.contains("DAB02570025"))]
/tmp/ipykernel_61671/3414202852.py:12: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .groupby(pd.Grouper(key="date_invoice", freq="M"))["quantity"]


In [28]:
ventas_mes

,date_invoice,quantity
0,2022-09-30,40.0
1,2022-10-31,34.0
2,2022-11-30,10.0
3,2022-12-31,13.0
4,2023-01-31,10.0
5,2023-02-28,66.0
6,2023-03-31,15.0
7,2023-04-30,24.0
8,2023-05-31,18.0
9,2023-06-30,14.0


In [29]:
prueba.info()

<class 'pandas.core.frame.DataFrame'>
Index: 276 entries, 46971 to 4579809
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   quantity                 276 non-null    float64       
 1   product_name             276 non-null    object        
 2   date_invoice             276 non-null    datetime64[ns]
 3   Sucursal                 276 non-null    object        
 4   producto_costo_unitario  276 non-null    float64       
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 12.9+ KB


In [33]:
import pandas as pd

# --- 1. Crear columna de Mes ---
prueba["Mes"] = prueba["date_invoice"].dt.to_period("M").dt.to_timestamp()

# --- 2. Agrupar por Sucursal + Producto + Mes ---
df_mensual = (prueba.groupby(["Sucursal", "product_name", "Mes"], as_index=False)
                     .agg({
                         "quantity": "sum",
                         "producto_costo_unitario": "last"
                     }))

# --- 3. Función para calcular EMA manual con α=0.2 ---
def calcular_ema(grupo, alpha=0.2):
    grupo = grupo.sort_values("Mes").reset_index(drop=True)
    ema0 = grupo.loc[:3, "quantity"].mean()  # EMA inicial
    ema_vals = [ema0]
    for t in range(1, len(grupo)):
        xt = grupo.loc[t, "quantity"]
        ema_prev = ema_vals[-1]
        ema_t = alpha * xt + (1 - alpha) * ema_prev
        ema_vals.append(ema_t)
    grupo["EMA"] = ema_vals
    return grupo


def calcular_ema_dual(grupo, alpha=0.2):
    grupo = grupo.sort_values("Mes").reset_index(drop=True)
    ema0 = grupo.loc[:3, "quantity"].mean()
    ema_classic = [ema0]
    ema_invert = [ema0]

    for t in range(1, len(grupo)):
        xt = grupo.loc[t, "quantity"]

        # EMA clásico
        ema_classic.append(alpha * xt + (1 - alpha) * ema_classic[-1])

        # EMA invertido
        ema_invert.append((1 - alpha) * xt + alpha * ema_invert[-1])

    grupo["EMA_clasico"] = ema_classic
    grupo["EMA_invertido"] = ema_invert
    return grupo



df_mensual = df_mensual.groupby(["Sucursal", "product_name"], group_keys=False).apply(calcular_ema_dual)

# --- 4. Columnas Año y Semana ---
df_mensual["Año"] = df_mensual["Mes"].dt.year

# --- 5. Ajuste de nombres de columnas como en tu sistema ---
df_final = df_mensual.rename(columns={
    "Sucursal": "Sucursal",
    "product_name": "Producto",
    "producto_costo_unitario": "Costo unitario"
})

# Aquí deberías enlazar también tus catálogos de productos
# para agregar "Descripción" y "Código producto"
# (ej. con un merge si tienes otro DataFrame de catálogo)

df_final.sort_values(by="Mes")




/tmp/ipykernel_61671/681169622.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prueba["Mes"] = prueba["date_invoice"].dt.to_period("M").dt.to_timestamp()
/tmp/ipykernel_61671/681169622.py:48: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_mensual = df_mensual.groupby(["Sucursal", "product_name"], group_keys=False).apply(calcular_ema_dual)


,Sucursal,Producto,Mes,quantity,Costo unitario,EMA_clasico,EMA_invertido,Año
0,BARRANQUILLA,[DAB02570025] DA2570 FILTRO AIRE DONSSON - PER...,2022-09-01,40.0,13507.71,24.250000,24.250000,2022
1,BARRANQUILLA,[DAB02570025] DA2570 FILTRO AIRE DONSSON - PER...,2022-10-01,34.0,13507.71,26.200000,32.050000,2022
2,BARRANQUILLA,[DAB02570025] DA2570 FILTRO AIRE DONSSON - PER...,2022-11-01,10.0,13507.71,22.960000,14.410000,2022
3,BARRANQUILLA,[DAB02570025] DA2570 FILTRO AIRE DONSSON - PER...,2022-12-01,13.0,13507.71,20.968000,13.282000,2022
4,BARRANQUILLA,[DAB02570025] DA2570 FILTRO AIRE DONSSON - PER...,2023-01-01,10.0,13507.71,18.774400,10.656400,2023
5,BARRANQUILLA,[DAB02570025] DA2570 FILTRO AIRE DONSSON - PER...,2023-02-01,66.0,13507.71,28.219520,54.931280,2023
6,BARRANQUILLA,[DAB02570025] DA2570 FILTRO AIRE DONSSON - PER...,2023-03-01,15.0,13507.71,25.575616,22.986256,2023
7,BARRANQUILLA,[DAB02570025] DA2570 FILTRO AIRE DONSSON - PER...,2023-04-01,24.0,13507.71,25.260493,23.797251,2023
8,BARRANQUILLA,[DAB02570025] DA2570 FILTRO AIRE DONSSON - PER...,2023-05-01,18.0,13507.71,23.808394,19.159450,2023
9,BARRANQUILLA,[DAB02570025] DA2570 FILTRO AIRE DONSSON - PER...,2023-06-01,14.0,13507.71,21.846715,15.031890,2023
